## Recommendation using Matrix Factorization

Recommends hotels on the Las Vegas Strip for the selected traveler type, based on 2015 TripAdvisor ratings.

### NuGet packages

In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.Recommender"
#r "nuget:XPlot.Plotly"

Installing package Microsoft.ML.................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package Microsoft.ML.Recommender......done!

Successfully added reference to package Microsoft.ML.Recommender, version 0.16.0

Installing package XPlot.Plotly.....done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

### Namespaces

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using XPlot.Plotly;
using System.IO;
using System.Linq;

### Input class definition

In [3]:
public class RecommendationData
{
    public float Label;

    public string TravelerType;

    public string Hotel;
}

### Output class definition

In [4]:
public class RecommendationPrediction
{
    public float Score;

    public string TravelerType;

    public string Hotel;
}

### Read the raw data

In [5]:
var mlContext = new MLContext(seed: null);

// Populating an IDataView from an IEnumerable.
var allData = File.ReadAllLines("./LasVegasTripAdvisorReviews.csv")
    .Skip(1)
    .Select(x => x.Split(';'))
    .Select(x => new RecommendationData
    {
        Label = uint.Parse(x[4]),
        TravelerType = x[6],
        Hotel = x[13]
    });
    
var data = allData
    .OrderBy(x => (x.GetHashCode())) // Cheap Randomization.
    .Take(400);

var trainingData = mlContext.Data.LoadFromEnumerable(data);

// Keep in memory.
trainingData = mlContext.Data.Cache(trainingData);

// ? doing nothing with this ?
// var dataView = mlContext.Data.CreateEnumerable<RecommendationData>(trainingData, reuseRowObject: false);

### Prepare the data

In [6]:
var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Hotel")
                .Append(mlContext.Transforms.Conversion.MapValueToKey("TravelerType"))
                .Append(mlContext.Recommendation().Trainers.MatrixFactorization(
                                    labelColumnName: "Label",
                                    matrixColumnIndexColumnName: "Hotel",
                                    matrixRowIndexColumnName: "TravelerType",
                                    // Optional fine tuning:
                                    numberOfIterations: 20,
                                    approximationRank: 8,
                                    learningRate: 0.4))
                .Append(mlContext.Transforms.Conversion.MapKeyToValue("Hotel"))
                .Append(mlContext.Transforms.Conversion.MapKeyToValue("TravelerType"));

### Train the model

In [7]:
var model = pipeline.Fit(trainingData);

### Evaluate the model

In [8]:
data = allData
    .OrderBy(x => (x.GetHashCode())) // Cheap Randomization.
    .TakeLast(200);

var testData = mlContext.Data.LoadFromEnumerable(data);
var scoredData = model.Transform(testData);
var metrics = mlContext.Recommendation().Evaluate(scoredData);

display(metrics);

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.8971147203445434,1.2153311409609484,1.102420582609445,1.2153311384102563,-0.17095205796410906


### Values

In [9]:
var hotels = allData.Select(r => r.Hotel).Distinct().ToList();
hotels.Sort();
display (hotels);

var travelerTypes = allData.Select(r => r.TravelerType).Distinct().ToList();
travelerTypes.Sort();
display (travelerTypes);

index,value
0,Bellagio Las Vegas
1,Caesars Palace
2,Circus Circus Hotel & Casino Las Vegas
3,Encore at wynn Las Vegas
4,Excalibur Hotel & Casino
5,Hilton Grand Vacations at the Flamingo
6,Hilton Grand Vacations on the Boulevard
7,Marriott's Grand Chateau
8,Monte Carlo Resort&Casino
9,Paris Las Vegas


index,value
0,Business
1,Couples
2,Families
3,Friends
4,Solo


### Single prediction

In [10]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<RecommendationData, RecommendationPrediction>(model);

var prediction = predictionEngine.Predict(new RecommendationData
    { 
        TravelerType = "Business",
        Hotel = "The Cromwell"
    });
display(prediction);

Score,TravelerType,Hotel
3.6061778,Business,The Cromwell


### Bulk prediction

Predicts the score for all hotels, for one specific traveler type.

In [11]:
var input = hotels.Select(h => new RecommendationData
               {
                   TravelerType = "Families",
                   Hotel = h
               }).ToList();
var predictions = model.Transform(mlContext.Data.LoadFromEnumerable(input));
var result = mlContext.Data.CreateEnumerable<RecommendationPrediction>(predictions, reuseRowObject: false);
result = result.OrderByDescending(r => r.Score);
display(result);

index,Score,TravelerType,Hotel
0,4.60616,Families,Wynn Las Vegas
1,4.4763727,Families,Tropicana Las Vegas - A Double Tree by Hilton Hotel
2,4.4739547,Families,Tuscany Las Vegas Suites & Casino
3,4.3850675,Families,The Westin las Vegas Hotel Casino & Spa
4,4.3496294,Families,The Cosmopolitan Las Vegas
5,4.3178434,Families,The Palazzo Resort Hotel Casino
6,4.2765317,Families,Paris Las Vegas
7,4.2595563,Families,The Venetian Las Vegas Hotel
8,4.201662,Families,Monte Carlo Resort&Casino
9,4.150102,Families,Hilton Grand Vacations at the Flamingo


### Visualization

In [12]:
var graph = new Graph.Bar()
{
    y = result.Select(r => r.Score),
    x = result.Select(r => r.Hotel),
    marker = new Graph.Marker { color = "darkred" }
};

var chart = Chart.Plot(graph);

var layout = new Layout.Layout(){ title="Recommended Hotels for Families" };
chart.WithLayout(layout);

display(chart);